# 2: Dataset Collectiona and Preparation

Import Neccessary Libraries

In [ ]:
import csv
import os
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
num_coords = len(results.pose_landmarks.landmark)
num_coords

Body Angle Calculation function using Pose Landmarks

In [ ]:
#Function for Angle calculation
def calculateAngle (landmark1,landmark2, landmark3):
    #Get the required landmarks coord.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    #Calculate the angle between the three points.
    angle = math.degrees(math.atan2(y3-y2, x3-x2)- math.atan2(y1-y2, x1-x2))
    #Check if angle is < 0
    if angle < 0:
        #add 360 to get positive corresponding angle
        angle += 360

    #return calculated angle 
    return angle 

#Function for distance calculation
def distance_between_landmarks(landmark1, landmark2):
    """Calculates the Euclidean distance between two landmark coordinates.

    Args:
        landmark1: A tuple or list containing the x and y coordinates of the first landmark.
        landmark2: A tuple or list containing the x and y coordinates of the second landmark.

    Returns:
        The Euclidean distance between the two landmarks.
    """

    x1, y1, _ = landmark1
    x2, y2, _= landmark2

    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    return distance



View inference function using Pose Landmarks

In [ ]:
def infer_viewpoint(landmarks):
    """
    Infers the viewpoint of a person based on their pose.

    Args:
        landmarks: A list of MediaPipe pose landmarks.

    Returns:
        A string representing the inferred viewpoint ("Front" or "Side").
    """
    
    if all(landmark.visibility > landmark_visibility_threshold for landmark in results.pose_landmarks.landmark):

        # Extract relevant landmarks

        #INDEX FOR RIGHT PART OF THE BODY
        right_elbow_index = mp_pose.PoseLandmark.RIGHT_ELBOW
        right_shoulder_index = mp_pose.PoseLandmark.RIGHT_SHOULDER
        right_hip_index = mp_pose.PoseLandmark.RIGHT_HIP
        right_wrist_index = mp_pose.PoseLandmark.RIGHT_WRIST
        right_heel_index = mp_pose.PoseLandmark.RIGHT_HEEL
        right_knee_index = mp_pose.PoseLandmark.RIGHT_KNEE
        right_ankle_index = mp_pose.PoseLandmark.RIGHT_ANKLE
        #INDEX FOR LEFT PART OF THE BODY
         
        left_elbow_index = mp_pose.PoseLandmark.LEFT_ELBOW
        left_shoulder_index = mp_pose.PoseLandmark.LEFT_SHOULDER
        left_hip_index = mp_pose.PoseLandmark.LEFT_HIP
        left_wrist_index = mp_pose.PoseLandmark.LEFT_WRIST
        left_heel_index = mp_pose.PoseLandmark.LEFT_HEEL
        left_knee_index = mp_pose.PoseLandmark.LEFT_KNEE
        left_foot_index = mp_pose.PoseLandmark.LEFT_FOOT_INDEX

        left_shoulder= (int(results.pose_landmarks.landmark[left_shoulder_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_shoulder_index].y * image.shape[0]),
                            int(results.pose_landmarks.landmark[left_shoulder_index].y * image.shape[0]), results.pose_landmarks.landmark[left_shoulder_index].visibility)
        right_shoulder= (int(results.pose_landmarks.landmark[right_shoulder_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_shoulder_index].y * image.shape[0]),
                            int(results.pose_landmarks.landmark[right_shoulder_index].y * image.shape[0]), results.pose_landmarks.landmark[right_shoulder_index].visibility)
        left_hip= (int(results.pose_landmarks.landmark[left_hip_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_hip_index].y * image.shape[0]),
                            int(results.pose_landmarks.landmark[left_hip_index].y * image.shape[0]), results.pose_landmarks.landmark[left_hip_index].visibility)
        right_hip= (int(results.pose_landmarks.landmark[right_hip_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_hip_index].y * image.shape[0]),
                            int(results.pose_landmarks.landmark[right_hip_index].y * image.shape[0]), results.pose_landmarks.landmark[right_hip_index].visibility)
        left_foot = (int(results.pose_landmarks.landmark[left_foot_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_foot_index].y * image.shape[0]),
                            int(results.pose_landmarks.landmark[left_foot_index].y * image.shape[0]), results.pose_landmarks.landmark[left_foot_index].visibility)
        left_heel = (int(results.pose_landmarks.landmark[left_heel_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_heel_index].y * image.shape[0]),
                            int(results.pose_landmarks.landmark[left_heel_index].y * image.shape[0]), results.pose_landmarks.landmark[left_heel_index].visibility)
        
        # Calculate distances
        shoulder_distance = np.linalg.norm(np.array([left_shoulder[0] - right_shoulder[0], left_shoulder[1] - right_shoulder[1], left_shoulder[2] - right_shoulder[2]]))
        hip_distance = np.linalg.norm(np.array([left_hip[0] - right_hip[0], left_hip[1] - right_hip[1], left_hip[2] - right_hip[2]]))
        foot_distance = np.linalg.norm(np.array([left_foot[0] - left_heel[0], left_foot[1] - left_heel[1], left_foot[2] - left_heel[2]]))

        # Analyze depth information
        depth_difference = left_shoulder[2] - right_shoulder[2]
        print(depth_difference)

        # Infer viewpoint
        if shoulder_distance > foot_distance or hip_distance > foot_distance :
            viewpoint = "Front"        
        else:
            viewpoint = "Side"
        
    return viewpoint

Reference Point calculation function using Pose Landmarks

In [ ]:
def calculate_square_center(corner1, corner2, corner3, corner4):
    """
    Calculate the center of a square given the coordinates of its corners.
    
    Parameters:
    - corner1, corner2, corner3, corner4: Tuples representing the coordinates of the square's corners (x, y).
    
    Returns:
    - A tuple representing the coordinates of the center (x, y).
    """
    # Sum up the x and y coordinates of all corners
    x_sum = corner1[0] + corner2[0] + corner3[0] + corner4[0]
    y_sum = corner1[1] + corner2[1] + corner3[1] + corner4[1]
    
    # Divide by 4 to get the average coordinates (center)
    center_x = x_sum / 4
    center_y = y_sum / 4
    center_z =0
    
    return (center_x, center_y, center_z )



Capture landmark data for creating dataset from video of person walking and save in .csv file format
        --DO THIS FOR ALL PERSONS INDICATING EACH VIEW BEING COLLECTED--

In [ ]:
cap = cv2.VideoCapture("C:\\Users\\FAVOUR\\Pictures\\project pic\\Stud_C\\VID-20241021-WA0024.mp4")

#Define class name and view
class_name = "Chijioke"
viewpoint1="Side"

# Frame skipping parameter
frame_skip = 5
frame_count = 0


# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames
        frame_count += 1 
        if frame_count % frame_skip != 0:
            continue
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        

        # Make Detections
        results = holistic.process(image)
        id= str(results.pose_landmarks)
        landmark_visibility_threshold = 0.1

        if not id == "None":          
            # Recolor image back to BGR for rendering
            image.flags.writeable = True   
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #Pose Detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                    )
            #Getting pose detection landmarks and coordinates
            if results.pose_landmarks:
                #Get the landmark index for body parts
                #right_sholder_index = 12// left_shoulder_index = 11
                #right_elbow_index = 14 // left_elbow_index =13
                #right_hip_index= 24 // left_hip_index =23
                #right_wrist_index = 16 // left_wrist_index = 15
                #right_heel_index = 30 // left_heel_heel = 29
                #right_knee_index = 26 // left_knee_index = 25
            
                #Index for right body part
                right_elbow_index = mp_pose.PoseLandmark.RIGHT_ELBOW
                right_shoulder_index = mp_pose.PoseLandmark.RIGHT_SHOULDER
                right_hip_index = mp_pose.PoseLandmark.RIGHT_HIP
                right_wrist_index = mp_pose.PoseLandmark.RIGHT_WRIST
                right_heel_index = mp_pose.PoseLandmark.RIGHT_HEEL
                right_knee_index = mp_pose.PoseLandmark.RIGHT_KNEE
                right_ankle_index = mp_pose.PoseLandmark.RIGHT_ANKLE
               
                #Index for left body part
                left_elbow_index = mp_pose.PoseLandmark.LEFT_ELBOW
                left_shoulder_index = mp_pose.PoseLandmark.LEFT_SHOULDER
                left_hip_index = mp_pose.PoseLandmark.LEFT_HIP
                left_wrist_index = mp_pose.PoseLandmark.LEFT_WRIST
                left_heel_index = mp_pose.PoseLandmark.LEFT_HEEL
                left_knee_index = mp_pose.PoseLandmark.LEFT_KNEE
                left_ankle_index = mp_pose.PoseLandmark.LEFT_ANKLE
                # Get the x and y coordinates of the left body part
                left_elbow_cord= (int(results.pose_landmarks.landmark[left_elbow_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_elbow_index].y * image.shape[0]), 0)
                left_shoulder_cord= (int(results.pose_landmarks.landmark[left_shoulder_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_shoulder_index].y * image.shape[0]),0)
                left_hip_cord= (int(results.pose_landmarks.landmark[left_hip_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_hip_index].y * image.shape[0]),0)
                left_wrist_cord= (int(results.pose_landmarks.landmark[left_wrist_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_wrist_index].y * image.shape[0]),0)
                left_heel_cord= (int(results.pose_landmarks.landmark[left_heel_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_heel_index].y * image.shape[0]),0)
                left_knee_cord= (int(results.pose_landmarks.landmark[left_knee_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_knee_index].y * image.shape[0]),0)
                left_ankle_cord= (int(results.pose_landmarks.landmark[left_ankle_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_ankle_index].y * image.shape[0]),0)
                left_hand_cord= (int(results.pose_landmarks.landmark[15].x * image.shape[1]), int(results.pose_landmarks.landmark[15].y * image.shape[0]),0)
                #Get the x and y coordinates of the right body part
                right_elbow_cord= (int(results.pose_landmarks.landmark[right_elbow_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_elbow_index].y * image.shape[0]), 0)
                right_shoulder_cord= (int(results.pose_landmarks.landmark[right_shoulder_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_shoulder_index].y * image.shape[0]),0)
                right_hip_cord= (int(results.pose_landmarks.landmark[right_hip_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_hip_index].y * image.shape[0]),0)
                right_wrist_cord= (int(results.pose_landmarks.landmark[right_wrist_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_wrist_index].y * image.shape[0]),0)
                right_heel_cord= (int(results.pose_landmarks.landmark[right_heel_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_heel_index].y * image.shape[0]),0)
                right_knee_cord= (int(results.pose_landmarks.landmark[right_knee_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_knee_index].y * image.shape[0]),0)
                right_ankle_cord= (int(results.pose_landmarks.landmark[right_ankle_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_ankle_index].y * image.shape[0]),0)
                right_hand_cord= (int(results.pose_landmarks.landmark[16].x * image.shape[1]), int(results.pose_landmarks.landmark[16].y * image.shape[0]),0)
                #Calculate the angles 
                right_arm_angle2 = calculateAngle(right_shoulder_cord, right_elbow_cord, right_hip_cord)
                left_arm_angle2= calculateAngle(left_shoulder_cord, left_elbow_cord, left_hip_cord)
                right_arm_angle1=calculateAngle(right_shoulder_cord, right_elbow_cord, right_wrist_cord)
                left_arm_angle1=calculateAngle(left_shoulder_cord, left_elbow_cord, left_wrist_cord)
                right_leg_angle1 =calculateAngle(right_shoulder_cord, right_hip_cord, right_knee_cord)
                left_leg_angle1 =calculateAngle(left_shoulder_cord, left_hip_cord, left_knee_cord)
                right_leg_angle2 =calculateAngle(right_hip_cord, right_knee_cord, right_ankle_cord)
                left_leg_angle2 =calculateAngle(left_hip_cord, left_knee_cord, left_ankle_cord)

                #distance between the heels
                heel_dist = distance_between_landmarks(right_heel_cord, left_heel_cord)
                                                    
                #distance between the wrists
                wrist_dist= distance_between_landmarks(right_wrist_cord, left_wrist_cord)
            
                #distance between the knee
                knee_dist= distance_between_landmarks(right_knee_cord, left_knee_cord)
                #height
                height= distance_between_landmarks(right_elbow_cord,right_heel_cord)
                #reference point
                center =  calculate_square_center(right_elbow_cord,left_elbow_cord,right_hip_cord,left_hip_cord)  
                R_heel_ref= distance_between_landmarks(left_heel_cord, center)  
                L_heel_ref= distance_between_landmarks(right_heel_cord,center)
                R_hand_ref= distance_between_landmarks(right_hand_cord, center)
                L_hand_ref= distance_between_landmarks(left_hand_cord, center)
                print(center, R_hand_ref,R_heel_ref)
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
                                
            if all(landmark.visibility > landmark_visibility_threshold for landmark in results.pose_landmarks.landmark):
                viewpoint = infer_viewpoint(results.pose_landmarks)
                print(f"Viewpoint: {viewpoint}")
                # Display Class
                cv2.putText(image, 'VIEW'
                    , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, viewpoint.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            else:
                print("Not all landmarks detected.")    
            #Concate rows
            row = []

            #Append class name 
            row.insert(0, class_name)
            row.insert(1, viewpoint1)
            row.insert(2, heel_dist)
            row.insert(3, wrist_dist)
            row.insert(4, knee_dist)
            row.insert(5, right_arm_angle1)
            row.insert(6, left_arm_angle1)
            row.insert(7, right_arm_angle2)
            row.insert(8, left_arm_angle2)
            row.insert(9, right_arm_angle1)
            row.insert(10, left_leg_angle1)
            row.insert(11, right_leg_angle2)
            row.insert(12, left_leg_angle2)
            row.insert(13, height)
            row.insert(14, R_hand_ref)
            row.insert(15, L_hand_ref)
            row.insert(16, R_heel_ref)
            row.insert(17, L_heel_ref)                       

            #Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)                  
             
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

Load .csv file as pandas dataframe and add the names of the columns

In [ ]:
df= pd.read_csv("C:\\Users\\FAVOUR\\Documents\\AI\\Gait detection research\\My_project_Gait_detection\\coords copy.csv")
columns=['Class','view','heel_dist','wrist_dist','knee_dist','right_arm_angle1','left_arm_angle1','right_arm_angle2','left_arm_angle2','right_leg_angle1','left_leg_angle1','right_leg_angle2','left_leg_angle2','Height','R_hand_ref','L_hand_ref', 'R_heel_ref','L_heel_ref']
# Assigning column names
df.columns = columns
# Writing the CSV file with the new column names
df.to_csv('coords.csv', index=False)


Plot features to visually analyse movement pattern

In [ ]:
l=[]
for i in range(len(df)):
    l.append(i)
print(l)

num_columns = len(df.columns) - 1  # Exclude 'X' column
fig, axs = plt.subplots(num_columns, 1, figsize=(8, 4 * num_columns))

# Flatten axs in case it's a single subplot
if num_columns == 1:
    axs = [axs]

for i, column in enumerate(df.columns[1:]):  # Skipping the 'X' column
    axs[i].plot(l, df[column])
    axs[i].set_title(f'Plot of {column} vs Y')